In [88]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import display, clear_output
import time
import os
import datetime

In [90]:
class Car:
    """
    This is a simple car model
    """
    def __init__(self,car_id,init_s=0.0,init_v=0.0):
        self.id = car_id
        self.s = init_s
        self.init_s = init_s
        # displacement, save itit value for rest
        
        self.v = init_v
        self.init_v = init_v
        # velocity, save itit value for rest
        
        self.a = 0.0 
        # acceleration

        self.s_history = [self.s]
        self.v_history = [self.v]
        self.a_history = [self.a]
        # arreys to save data
    
    def reset(self):
        self.s = self.init_s
        self.v = self.init_v
        self.a = 0.0
     
        self.s_history = [self.s]
        self.v_history = [self.v]
        self.a_history = [0.0]
        # reset to initial values
        
    def set_a(self, a):
        self.a = a
        # get acceleration
        """
        accel is the system input of the auto car project
        the accel of ego car will be defined by NN
        the accels of others car will be generate by some defined rule
        for example, randomly
        """

    def step(self, dt):
        self.s += self.v*dt+0.5*self.a*dt*dt
        self.v += dt*self.a

        self.s_history.append(self.s)
        self.v_history.append(self.v)
        self.a_history.append(self.a)
        """ 
        save all history of displacement (s) and velocity (v) 
        into the arrey self.s_history and self.v_history
        """
        
        

In [91]:
class PID_controller:
    
    """
    This is a very simple PID control logic
    """
    def __init__(self,Kp,Ki,Kd,dt):
        """
        This is the PID init
        Kp for proportional gain
        Ki for Integral gain
        Kd for Dericative gain
        dt for the time inerval
        err_integral is the add up err from the begaining
        err_previous is the err from the previous time step
        """
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.dt = dt
        self.err_integral = 0.0
        self.err_previous = 0.0
        self.output_history = [0.0]
        self.err_history = [0.0]

    def step(self,err):
        
        # P control
        p_out = self.Kp*err

        # I control
        i_out = self.Ki*self.err_integral
        # update the addup err
        self.err_integral += err*dt 

        # D control
        d_out = self.Kd*((err-self.err_previous)/self.dt)
        # record current err for next time step 
        # will become previous err in next time step
        self.err_previous = err

        control_output = p_out + i_out + d_out
        self.output_history.append(control_output)
        self.err_history.append(err)
        return control_output

    def reset(self):

        # init the controller status before using
        self.err_integral = 0.0
        self.err_previous = 0.0
        self.output_history = [0.0]
        self.err_history = [0.0]
        

In [92]:
def run_single_simulation(simulation_time,dt):
    # simulation_time = 200.0
    # dt = 0.1
    np.random.seed()
    # np.random.seed(24)
    
    random_gap = np.random.uniform(20,30)
    random_v = np.random.uniform(10,15)
    
    ego_car = Car(car_id="ego",init_s=0.0,init_v=random_v)
    #generate ego car at initial point with random velocity
    
    front_car = Car(car_id="front",init_s=2*random_gap,init_v=random_v)
    back_car = Car(car_id="back",init_s=-random_gap,init_v=random_v)
    #generate the front car and the back car as well
    
    cars = [ego_car, front_car, back_car]
    
    #print(f"New cars are created!")
    #print(f"The ego car is at {ego_car.s} m, with v={ego_car.v} m/s and a={ego_car.a} m/s^2")
    #print(f"The front car is at {front_car.s} m, with v={front_car.v} m/s and a={front_car.a} m/s^2")
    #print(f"The back car is at {back_car.s} m, with v={back_car.v} m/s and a={back_car.a} m/s^2")
    
    
    
    
    """
    !!!!!!!!!!!
    Here are all the init params to be defined
    
    I should put them somewhere outside this func
    I'll find time to modify, trust me
    ----Ocean
    """
    
    a_set = 0.0
    # set ego car accel
    a_set_random = 0.0
    # set random accel for front and back car
    # using a_set_max to limit the amplitude
    
    a_set_max = 3.0
    # Max accel for ego car (speed up)
    a_set_min = -5.0
    # Min accel for ego car (
    
    d_diff = 0.0 
    # also considered d_diff as error for the controller
    
    Kp = 0.085
    Ki = 0.00002955
    Kd = 0.070
    # select control params, better not change any of them
    
    a_accept = 1.0
    """
    if the accel chosen the by the controller 
    within the range of [-a_accept,a_accept]
    the ego car won't take any action
    """
    
    #a_step_disterbance = 0.1
    
    ego_controller = PID_controller(Kp,Ki,Kd,dt)
    
    d_2_front = [2*random_gap]
    d_2_back = [random_gap]
    v_2_front = [0.0]
    v_2_back = [0.0]
    
    # reset all cars and controller
    for car in cars:
        car.reset()
    ego_controller.reset
    
    
    
    time_steps = np.arange(0.0,simulation_time,dt)
    
    for k in range(len(time_steps)):
        d_diff = d_2_front[-1]-d_2_back[-1]
        """
        d_diff positive means the car is more closer to the back
        setting this way to make controller output and error 
        appear in the same side of Y-axis
        """
        a_set = ego_controller.step(d_diff)
        
        if a_set <= -a_accept:
            a_set = a_set_min
        elif a_set >= a_accept:
            a_set = a_set_max
        else:
            a_set = 0.0
        
        ego_car.set_a(a_set)
    
        np.random.seed()
        a_set_random = np.random.uniform(-a_set_max,a_set_max)
        front_car.set_a(a_set_random)
        back_car.set_a(a_set_random)
        
        ego_car.step(dt)
        front_car.step(dt)
        back_car.step(dt)
    
        d_2_front.append(front_car.s-ego_car.s)
        d_2_back.append(ego_car.s-back_car.s)
        v_2_front.append(front_car.v-ego_car.v)
        v_2_back.append(ego_car.v-back_car.v)
    
        if d_2_front[-1]<=0 or d_2_back[-1]<=0:
            #print("Here's a car crash!!! ")
            break
    
    
        
    
    print(f"Simulation finished!")
    
    # plot_time_axis = np.arange(len(ego_car.s_history))*dt
    
    # fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    # axes[0].set_title('Car Position', fontsize=16)
    # axes[0].set_xlabel('Time (s)', fontsize=12)
    # axes[0].set_ylabel('Position (m)', fontsize=12)
    # axes[0].plot(plot_time_axis, ego_car.s_history, label='Ego Car', color='red', linewidth=2)
    # axes[0].plot(plot_time_axis, front_car.s_history, label='Front Car', color='blue', linestyle='--')
    # axes[0].plot(plot_time_axis, back_car.s_history, label='Back Car', color='green', linestyle='-.')
    # axes[0].legend()
    # axes[0].grid(True)
    
    # axes[1].set_title('Ego Car Accel', fontsize=16)
    # axes[1].set_xlabel('Time (s)', fontsize=12)
    # axes[1].set_ylabel('Acceleration (m/s^2)', fontsize=12)
    # axes[1].plot(plot_time_axis, ego_car.a_history, label='Ego Car Acceleration', color='purple')
    # axes[1].plot(plot_time_axis, ego_controller.err_history, label='distance to mid-point', color='red')
    
    # axes[1].legend()
    # axes[1].grid(True)
    # plt.show() 
    
    return {
        "d_front": d_2_front,
        "d_back": d_2_back,
        "v_front": v_2_front,
        "v_back": v_2_back,
        "ego_a": ego_car.a_history
    }


In [93]:
num_run = 10
simulation_time = 100
dt = 0.1

all_data = []

print(f"Start to generate data, running {num_run} times")

# --- run simulations ---
for i in range(num_run):
    print(f"Running for {i + 1}/{num_run} time...")
    single_run_data = run_single_simulation(simulation_time, dt)
    all_data.append(single_run_data)

print("Dataset generated！")

Start to generate data, running 10 times
Running for 1/10 time...
Simulation finished!
Running for 2/10 time...
Simulation finished!
Running for 3/10 time...
Simulation finished!
Running for 4/10 time...
Simulation finished!
Running for 5/10 time...
Simulation finished!
Running for 6/10 time...
Simulation finished!
Running for 7/10 time...
Simulation finished!
Running for 8/10 time...
Simulation finished!
Running for 9/10 time...
Simulation finished!
Running for 10/10 time...
Simulation finished!
Dataset generated！


In [94]:
processed_data = []
for run_id, run_data in enumerate(all_data):

    num_steps = len(run_data['ego_a'])
    
    for step in range(num_steps):

        record = {
            'run_id': run_id,
            'time_step': step,
            'd_front': run_data['d_front'][step],
            'd_back': run_data['d_back'][step],
            'v_front': run_data['v_front'][step],
            'v_back': run_data['v_back'][step],
            'ego_a': run_data['ego_a'][step]
        }
        processed_data.append(record)

df_to_save = pd.DataFrame(processed_data)
current_dir = os.getcwd()

#print(current_dir)

output_folder = os.path.join(current_dir, 'car-manual-dataset')
os.makedirs(output_folder, exist_ok=True)

timestamp_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
file_path = os.path.join(output_folder, f"simulation-dataset-{timestamp_str}.csv")

# try:
#     df_to_save.to_csv(file_path, index=False, encoding='utf-8-sig')
#     print(f"dataset saved at{file_path}")
# except Exception as e:
#     print(f"saving dateset error: {e}")
